### Package Imports & Path Setup

In [1]:
import sys
import os

In [2]:
import pandas as pd

In [4]:
#enabling importing of modules from utils directory

# construct path to utils directory
path_to_utils = os.path.abspath(os.path.join(os.path.dirname('__file__'), 'utils')) 

# avoid adding multiple times
if path_to_utils not in sys.path:
    sys.path.append(path_to_utils)

### Data Import

In [6]:
print(os.getcwd())

/Users/aoluwolerotimi/GitHubProjects/Kickstarter-Project-Clustering


In [7]:
data = pd.read_excel("./data/Kickstarter.xlsx")

### Data Profiling

### Testing, delete later

In [3]:
from kscleaning import testprint

In [4]:
result = testprint()
print(result)

function called
function executed
